In [71]:
import pandas as pd


def importar():
    """importa os dados necessários e devolve dois dados: tabela da malária e dados dos municípios
    arg: nenhum
    retunr: df =  tabela pandas para base de dados e municipios = tabela pandas dos municipios e da população por ano."""
    df = pd.read_pickle("dados\Dados_utilizaveis")
    # criar coluna com dados de casos confirmados como 1, outros resultados como 0
    df["detection.numerico"] = df["detection.type"].map(
        {"active": 1, "passive": 0, "CVL": 0}
    )
    municipio = pd.read_pickle("dados\municipio")
    municipio["id_municipio"] = (
        municipio["id_municipio"] // 10
    )  # Tirado ultimo digito do ID para facilitar seu uso, pois não possue diferencia retirar esse digito para o uso final.
    return df, municipio

In [72]:
import numpy as np
df,municipio=importar()
df.rename(columns = {'notification.year':'year', 'notification.month':'month'}, inplace = True)
df["day"]=np.ones(len(df))
df["data"]=pd.to_datetime(df[["year", "month", "day"]])
df=df.drop(columns=["month","year","day"])
dados_trimestrais=df.groupby([pd.Grouper(key="data",freq="Q"),"infection.county","occupation"]).sum()
logic1=dados_trimestrais["detection.numerico"]>0
dados_trimestrais=dados_trimestrais.loc[logic1].reset_index()
print(dados_trimestrais["data"].unique())
dados_trimestrais.head(20)

['2009-03-31T00:00:00.000000000' '2009-06-30T00:00:00.000000000'
 '2009-09-30T00:00:00.000000000' '2009-12-31T00:00:00.000000000'
 '2010-03-31T00:00:00.000000000' '2010-06-30T00:00:00.000000000'
 '2010-09-30T00:00:00.000000000' '2010-12-31T00:00:00.000000000'
 '2011-03-31T00:00:00.000000000' '2011-06-30T00:00:00.000000000'
 '2011-09-30T00:00:00.000000000' '2011-12-31T00:00:00.000000000'
 '2012-03-31T00:00:00.000000000' '2012-06-30T00:00:00.000000000'
 '2012-09-30T00:00:00.000000000' '2012-12-31T00:00:00.000000000'
 '2013-03-31T00:00:00.000000000' '2013-06-30T00:00:00.000000000'
 '2013-09-30T00:00:00.000000000' '2013-12-31T00:00:00.000000000'
 '2014-03-31T00:00:00.000000000' '2014-06-30T00:00:00.000000000'
 '2014-09-30T00:00:00.000000000' '2014-12-31T00:00:00.000000000'
 '2015-03-31T00:00:00.000000000' '2015-06-30T00:00:00.000000000'
 '2015-09-30T00:00:00.000000000' '2015-12-31T00:00:00.000000000'
 '2016-03-31T00:00:00.000000000' '2016-06-30T00:00:00.000000000'
 '2016-09-30T00:00:00.000

,data,infection.county,occupation,detection.numerico
0,2009-03-31,110001.0,ignored,4
1,2009-03-31,110002.0,agriculture,36
2,2009-03-31,110002.0,domestic worker,10
3,2009-03-31,110002.0,grazing,4
4,2009-03-31,110002.0,hunting and fishing,7
5,2009-03-31,110002.0,ignored,12
6,2009-03-31,110002.0,other,17
7,2009-03-31,110002.0,panning,25
8,2009-03-31,110002.0,traveler,1
9,2009-03-31,110008.0,agriculture,25


In [73]:
from sklearn.model_selection import train_test_split
TAMANHO_TESTE = 0.05

indices = dados_trimestrais.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE,  random_state = 41
)

df_treino = dados_trimestrais.loc[indices_treino]
df_teste = dados_trimestrais.loc[indices_teste]

print(df_treino)
print()
print(df_teste)

            data  infection.county            occupation  detection.numerico
16348 2014-12-31          140047.0                 other                  77
23427 2018-09-30          130180.0              traveler                   4
19529 2016-09-30          150280.0  vegetal exploitation                   9
17160 2015-06-30          130420.0               grazing                   1
941   2009-06-30          120042.0               ignored                   2
...          ...               ...                   ...                 ...
8513  2011-09-30          150120.0           agriculture                  92
21584 2017-12-31          120020.0              traveler                  10
20450 2017-03-31          150506.0       domestic worker                   6
931   2009-06-30          120039.0           agriculture                  11
1984  2009-09-30          130356.0           agriculture                  32

[25377 rows x 4 columns]

            data  infection.county           occu

In [99]:
from sklearn.preprocessing import OneHotEncoder
from seaborn import load_dataset

df = dados_trimestrais[['occupation']]
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df)
print(transformed.shape)
merge = (transformed.toarray())
print(merge)

(26713, 12)
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
